<a href="https://colab.research.google.com/github/gibranfp/CursoDatosMasivosI/blob/main/notebooks/2a_pyspark_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Primeros pasos

En este notebook haremos una revisión de los conceptos básicos de Spark con Python.

## 1.1 Introduccion

### _¿Qué es Apache Spark?_

Apache spark es un _framework_ de programación para el procesamiento de datos distribuidos (computacion en cluster) _open source_. Spark fue originalmente desarrolado por la Universidad de California Berkeley quien posteriomente donó el codigo del proyecto a la _Apache Software Foundation_, quien desde entonces se encarga de su mantenimiento.

Spark proporciona APIs para ser empleado en conjunto con lenguajes de programación como Java, Python, Scala y R. Además, cuenta con herramientas para la realización de tareas de alto nivel como Spark SQL (para el procesamiento de datos estructurados basado en SQL), MLlib (para la implementación de modelos de aprendizaje máquina), GraphX (para el procesamiento de grafos), etc.

### _¿Qué es PySpark?_

PySpark es una herramienta que nos deja usar Spark encima de Python. Permitiendonos combinar el proceso de datos distribuidos de Spark con la simplicidad de Python para el análisis de conjuntos masivos de datos (_big data_).

###¿Cómo usar PySpark en Google Colab?

Instalar PySpark y usarlo dentro de un Jupyter notebooks requiere de un poco más que solo correr `pip install pyspark` en la terminal. Para tratar de darle la vuelta al problema de la instalación de nuestro entorno de programación usaremos Colab, que nos proprociona un entorno _más controlado_ en el cual podemos instalar las bibliotecas necesarias de una forma más facil.

Sin embargo, si el lector requiere hacer una instalación de PySpark de forma local, los siguientes links pueden ser de ayuda:

* [macOS](https://medium.com/@yajieli/installing-spark-pyspark-on-mac-and-fix-of-some-common-errors-355a9050f735)
* [Windows](https://changhsinlee.com/install-pyspark-windows-jupyter/)
* [Linux](https://py.plainenglish.io/apache-spark-using-jupyter-in-linux-installation-and-setup-b2cacc6c7701)


En Colab podemos configurar nuestro ambiente de desarrollo simplemente ejecutando el contenido de la siguiente celda

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Para empezar a hacer uso de PySpark es necesario crear una `SparkSession`, dicho objeto es el que nos permitira crear RDDs o DataFrames, que son los objetos con los que más estaremos trabajando

In [ ]:
import pyspark
# Carga ufnciones extra
from pyspark.sql.functions import * 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('primeros_pasos').getOrCreate()

## 1.2 RDDs

La forma más elemental de trabajar con datos en PySpark es haciendo uso de un RDD (_Resilient Distributed Dataset_), el cual nos da la capacidad de manejar los datos y realizar cálculos con ellos de forma distribuida en varios nodos de un cluster.

Para realizar los ejemplos de este notebook haremos uso de una base de datos que cuenta con datos de los jugadores del juego FIFA20, dichos datos fueron tomados este [concurso de kaggle](https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset?select=players_20.csv) donde podrán encontrar una descripción detalalda de cada campo de la base datos.

Podemos crear un RDD a partir de un archivo CSV de la siguiente manera

In [ ]:
datos_crudos = spark.sparkContext.textFile('/content/drive/MyDrive/datos/players_20.csv')

Una de las desventajas de los RDDs es que solo proveen a los datos de la estructura minima para poder trabajar con ellos. Imprimamos los primeros dos elementos del RDD para ver en que condicion se encuentran nuestros datos

In [ ]:
for renglon in datos_crudos.take(2):
  print(renglon)

sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defe

Como podemos ver, los elementos del RDD son solo los renglones del archivo CSV sin formato. Siendo el primer elemento los nombres de los campos de la base de datos, el mantener a los nombres de los campos como valores dentro del RDD puede derivar en problemas más adelante, así que por el momento los removeremos para quedarnos solamente con los valores 

In [ ]:
# Guarda el primer elemento 
columnas = datos_crudos.take(1)[0]
# Guarda en todos los elementos de datos_crudos menos el primero correspondiente a las columnas
datos_crudos = datos_crudos.filter(lambda renglon : renglon != columnas)

Ahora podemos darle un poco más de estructura a nuestros datos. Primero separemos los campos de cada renglon, sabiendo que estan separados por comas, y quedandonos con solo un subconjunto de dichos campos, digamos... su nombre, edad, estatura, peso, nacionalidad, equipo y su ranking. Para esto definamos la siguiente función 

In [ ]:
def separar_campos(renglon):
    renglon_separado = renglon.split(',')
    # Extrae los campos que queremos
    nombre = str(renglon_separado[2])
    edad = int(renglon_separado[4])
    estatura = int(renglon_separado[6])
    peso = int(renglon_separado[7])
    nacionalidad = str(renglon_separado[8])
    equipo = str(renglon_separado[9])
    rank = int(renglon_separado[10])

    return (nombre, edad, estatura, peso, nacionalidad, equipo, rank)

aplicamos dicha función a cada renglon

In [ ]:
rdd_fifa = datos_crudos.map(separar_campos)

# Imprimimos una muestra de los datos
for renglon in rdd_fifa.take(5):
  print(renglon)

('L. Messi', 32, 170, 72, 'Argentina', 'FC Barcelona', 94)
('Cristiano Ronaldo', 34, 187, 83, 'Portugal', 'Juventus', 93)
('Neymar Jr', 27, 175, 68, 'Brazil', 'Paris Saint-Germain', 92)
('J. Oblak', 26, 188, 87, 'Slovenia', 'Atlético Madrid', 91)
('E. Hazard', 28, 175, 74, 'Belgium', 'Real Madrid', 91)


Hasta ahora, hemos realizado diferentes operaciones sobre nuestro RDD para poder darle un formato más tratable a los datos que contiene. Sin embargo, no hemos detallado nada sobre la naturaleza de dichas operaciones. Enfoquemonos ahora en hacer más explícitos estos detalles.

Existen dos operaciones básicas que pueden ser aplicadas a los RDD:

* Acciones: Son operaciones que accionan la realización de cálculos y regresan valores explícitos del RDD.

* Transformaciones: Son operaciones que devuelven otro RDD. Este tipo de operaciones no se ejecutan como tal sino hasta que es llamada una acción. Por esta razón se dice que las transformaciones son de evaluacion peresoza (_lazy evaluation_).

En el caso de la celda anterior, `map()` es una transformación que aplica la función dada a los elementos del RDD y `take()` es la acción que nos devuelve los $n$ primeros elementos.

Veamos más ejemplos. Podríamos querer saber cuantos jugadores existen por país en la base de datos. Para lograr esto primero necesitamos convertir nuestros datos en pares `(key, value)`, donde la `key`será el pais y el `value` nos indicará que ese registro pertenece a un jugador de dicho país 



In [ ]:
# Nos quedamos solo con la entrada corresponiente al país
rdd_nacionalidades = rdd_fifa.map(lambda renglon: (renglon[4], 1))

Usando los países como llaves y la transfromación llamada reduceByKey, sumaremos todos los valores para cada país.

In [ ]:
rdd_nacionalidades = rdd_nacionalidades.reduceByKey(lambda val_1 , val_2: val_1 + val_2)

Hasta el momento hemos usado las transformaciones `map()` y `reduceByKey()`, para obtener resultados necesitamos efectuar una acción, en este caso usaremos la acción `collect()`, que nos devuelve todo el contenido del RDD. Esta acción debe usarse con cuidado, en casos donde el RDD contenga una gran cantidad de registros, cargar todo el contenido del RDD puede dejar al sistema sin memoria.

In [ ]:
resultados = rdd_nacionalidades.collect()

for renglon in resultados[:10]:
  print(renglon)

('Argentina', 886)
('Portugal', 344)
('Brazil', 824)
('Slovenia', 61)
('Belgium', 268)
('Netherlands', 416)
('France', 984)
('Uruguay', 164)
('Poland', 324)
('Denmark', 345)


Aunque los resultados obtenidos son buenos, sería interesante poder ordenar a los paises por la cantidad de jugadores. En la siguiente celda imprimimos los 10 países con más jugadores registrados en el juego de FIFA20

In [ ]:
# Pedimos que ordene por total de jugadores
rdd_nacionalidades = rdd_nacionalidades.sortBy(lambda renglon: renglon[1], ascending = False)

print ("{:<12} {:<15}".format('PAIS', 'TOTAL JUGADORES'))

for renglon in rdd_nacionalidades.take(10):
  pais = renglon[0]
  total_jugadores = renglon[1]
  
  print ("{:<12} {:<15}".format(pais, total_jugadores))

PAIS         TOTAL JUGADORES
England      1667           
Germany      1216           
Spain        1035           
France       984            
Argentina    886            
Brazil       824            
Italy        732            
Colombia     591            
Japan        453            
Netherlands  416            


Además de las transformaciones y acciones empleadas aquí existen muchas más, sin embargo, los ejemplos expuestos aqui pueden ayudar al lector a familiarizarse con su uso.

Como el lector habrá notado, aunque los RDDs nos proveen de la capacidad de manipular y hacer calculos con datos distribuidos, éstos carecen de las funcionalidades y la estructura que tienen herramientas de un poco más alto nivel, como los DataFrame de Pandas, para el manejo de datos. 

En PySpark existe un tipo de objetos, igual llamados DataFrames, los cuales nos ayudan a dotar a nuestros datos con más funcionalidades y estructura, en comparación a los RDDs, al mismo tiempo que seguir trabajando de forma distribuida con ellos. Por esta razon, procederemos a dar una introducción a los DataFrames de PySpark que son los entes con los cuales más trabajaremos a lo largo del curso.

## 1.3 DataFrames

A diferencia de los RDDs, los DataFrames nos permiten estructurar a una collección de datos distribuidos en forma de tabla (con columnas y filas).

Podemos crear un DataFrame a partir de un archivo CSV de la siguiente manera

In [ ]:
df_fifa = spark.read.csv('/content/drive/MyDrive/datos/players_20.csv', header = True, inferSchema = True)

Para efectos de este tutorial no haremos uso de todos los campos de la base de datos, seleccionemos solo un subconjunto. Para esto usaremos la función `select()`, la cual devuelve un DataFrame con solo las columnas seleccionadas.

In [ ]:
df_fifa = df_fifa.select('short_name','age', 'height_cm', 'weight_kg', 'nationality', 'club', 'value_eur', 'preferred_foot')

Notece que, dada la estructrua que el DataFrame da a los datos, podemos hacer referencia a las columnas por su nombre en el archivo CSV.

### 1.3.1 Visualizar datos

Algo que es de vital importancia a la hora de manejar datos es poder visualizar, al menos, una pequeña muestra de éstos. Aquí algunas opciones.


In [ ]:
df_fifa.show(5)

+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|value_eur|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|         L. Messi| 32|      170|       72|  Argentina|       FC Barcelona| 95500000|          Left|
|Cristiano Ronaldo| 34|      187|       83|   Portugal|           Juventus| 58500000|         Right|
|        Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|105500000|         Right|
|         J. Oblak| 26|      188|       87|   Slovenia|    Atlético Madrid| 77500000|         Right|
|        E. Hazard| 28|      175|       74|    Belgium|        Real Madrid| 90000000|         Right|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



Si el contenido del DataFrame es lo suficientemente pequeño podemos transformar su contenido a un DataFrame de Pandas.

In [ ]:
df_pequeño = df_fifa.limit(10)
df_pandas = df_pequeño.toPandas()
df_pandas.head(5)

,short_name,age,height_cm,weight_kg,nationality,club,value_eur,preferred_foot
0,L. Messi,32,170,72,Argentina,FC Barcelona,95500000,Left
1,Cristiano Ronaldo,34,187,83,Portugal,Juventus,58500000,Right
2,Neymar Jr,27,175,68,Brazil,Paris Saint-Germain,105500000,Right
3,J. Oblak,26,188,87,Slovenia,Atlético Madrid,77500000,Right
4,E. Hazard,28,175,74,Belgium,Real Madrid,90000000,Right


Con Spark podemos concatenar las operaciones de la celda pasada de la siguiente forma.

In [ ]:
df_fifa.limit(10).toPandas().head(5)

,short_name,age,height_cm,weight_kg,nationality,club,value_eur,preferred_foot
0,L. Messi,32,170,72,Argentina,FC Barcelona,95500000,Left
1,Cristiano Ronaldo,34,187,83,Portugal,Juventus,58500000,Right
2,Neymar Jr,27,175,68,Brazil,Paris Saint-Germain,105500000,Right
3,J. Oblak,26,188,87,Slovenia,Atlético Madrid,77500000,Right
4,E. Hazard,28,175,74,Belgium,Real Madrid,90000000,Right


Aunque para ayudar a la legibilidad del codigo es comun separar las lineas largas asi

In [ ]:
df_fifa.limit(10) \
  .toPandas() \
  .head(5)

,short_name,age,height_cm,weight_kg,nationality,club,value_eur,preferred_foot
0,L. Messi,32,170,72,Argentina,FC Barcelona,95500000,Left
1,Cristiano Ronaldo,34,187,83,Portugal,Juventus,58500000,Right
2,Neymar Jr,27,175,68,Brazil,Paris Saint-Germain,105500000,Right
3,J. Oblak,26,188,87,Slovenia,Atlético Madrid,77500000,Right
4,E. Hazard,28,175,74,Belgium,Real Madrid,90000000,Right


Para poder revisar la organización de nuestro DataFrame y el `data tpye` de cada campo podemos usar lo siguiente

In [ ]:
df_fifa.printSchema()

root
 |-- short_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- nationality: string (nullable = true)
 |-- club: string (nullable = true)
 |-- value_eur: integer (nullable = true)
 |-- preferred_foot: string (nullable = true)



### 1.3.2 Funciones

#### **Renombar columnas**

Podemos cambiar el nombre de alguna columna 

In [ ]:
df_fifa.withColumnRenamed('value_eur','valor_euros').show(5)

+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|valor_euros|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
|         L. Messi| 32|      170|       72|  Argentina|       FC Barcelona|   95500000|          Left|
|Cristiano Ronaldo| 34|      187|       83|   Portugal|           Juventus|   58500000|         Right|
|        Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|  105500000|         Right|
|         J. Oblak| 26|      188|       87|   Slovenia|    Atlético Madrid|   77500000|         Right|
|        E. Hazard| 28|      175|       74|    Belgium|        Real Madrid|   90000000|         Right|
+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
only showing top 5 rows



Notemos que la llamada a la función no afectara al DataFrame `df_fifa`, para conservar los cambios efectuados debemos guardar la salida de la función en otro DataFrame

In [ ]:
df_ejemplo = df_fifa.withColumnRenamed('value_eur','valor_euros')
df_ejemplo.show(5)

+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|valor_euros|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
|         L. Messi| 32|      170|       72|  Argentina|       FC Barcelona|   95500000|          Left|
|Cristiano Ronaldo| 34|      187|       83|   Portugal|           Juventus|   58500000|         Right|
|        Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|  105500000|         Right|
|         J. Oblak| 26|      188|       87|   Slovenia|    Atlético Madrid|   77500000|         Right|
|        E. Hazard| 28|      175|       74|    Belgium|        Real Madrid|   90000000|         Right|
+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
only showing top 5 rows



#### **Descartar columnas**

Tambien podemos descartar columnas


In [ ]:
df_fifa.drop('weight_kg').show(5)

+-----------------+---+---------+-----------+-------------------+---------+--------------+
|       short_name|age|height_cm|nationality|               club|value_eur|preferred_foot|
+-----------------+---+---------+-----------+-------------------+---------+--------------+
|         L. Messi| 32|      170|  Argentina|       FC Barcelona| 95500000|          Left|
|Cristiano Ronaldo| 34|      187|   Portugal|           Juventus| 58500000|         Right|
|        Neymar Jr| 27|      175|     Brazil|Paris Saint-Germain|105500000|         Right|
|         J. Oblak| 26|      188|   Slovenia|    Atlético Madrid| 77500000|         Right|
|        E. Hazard| 28|      175|    Belgium|        Real Madrid| 90000000|         Right|
+-----------------+---+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



#### **Ordenar**

Podemos ordenar a los jugadores por su valor en euros


In [ ]:
df_fifa.orderBy('value_eur', ascending = False).show(5)

+------------+---+---------+---------+-----------+-------------------+---------+--------------+
|  short_name|age|height_cm|weight_kg|nationality|               club|value_eur|preferred_foot|
+------------+---+---------+---------+-----------+-------------------+---------+--------------+
|   Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|105500000|         Right|
|    L. Messi| 32|      170|       72|  Argentina|       FC Barcelona| 95500000|          Left|
|   K. Mbappé| 20|      178|       73|     France|Paris Saint-Germain| 93500000|         Right|
|K. De Bruyne| 28|      181|       70|    Belgium|    Manchester City| 90000000|         Right|
|   E. Hazard| 28|      175|       74|    Belgium|        Real Madrid| 90000000|         Right|
+------------+---+---------+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



#### **Filtrado con `where()`**

con la funcion `where()` podemos hacer un filtrado de nuestros datos. Por ejemplo, podemos querer tener solo la informacion de los jugadores mayores a 30 años

In [ ]:
df_fifa.where(df_ejemplo.age > 30).show(5)

+-----------------+---+---------+---------+-----------+---------------+---------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|           club|value_eur|preferred_foot|
+-----------------+---+---------+---------+-----------+---------------+---------+--------------+
|         L. Messi| 32|      170|       72|  Argentina|   FC Barcelona| 95500000|          Left|
|Cristiano Ronaldo| 34|      187|       83|   Portugal|       Juventus| 58500000|         Right|
|        L. Modrić| 33|      172|       66|    Croatia|    Real Madrid| 45000000|         Right|
|     G. Chiellini| 34|      187|       85|      Italy|       Juventus| 24500000|          Left|
|        S. Agüero| 31|      173|       70|  Argentina|Manchester City| 60000000|         Right|
+-----------------+---+---------+---------+-----------+---------------+---------+--------------+
only showing top 5 rows



tal vez quisieramos solo ver la informacion de los jugadores del Liverpool que tuviera mas de 30 años de edad

In [ ]:
df_fifa.where((df_ejemplo.age > 30) & (df_ejemplo.club == 'Liverpool')).show(5)

+-----------+---+---------+---------+-----------+---------+---------+--------------+
| short_name|age|height_cm|weight_kg|nationality|     club|value_eur|preferred_foot|
+-----------+---+---------+---------+-----------+---------+---------+--------------+
|  J. Milner| 33|      175|       70|    England|Liverpool| 10500000|         Right|
| A. Lallana| 31|      178|       73|    England|Liverpool| 11000000|         Right|
|     Adrián| 32|      190|       80|      Spain|Liverpool|  4700000|         Right|
|A. Lonergan| 35|      192|       87|    England|Liverpool|   150000|          Left|
+-----------+---+---------+---------+-----------+---------+---------+--------------+



#### **Funciones de agregación y `groupBy()`**

Supongamos que queremos saber el costo total en euros de un equipo, lograríamos dicho objetivo sumando el valor de los jugadores de cada equipo. Esto lo podemos lograr usando la funcion `groupBy()` en conjunto con algunas funciones de agregación de la siguiente manera

In [ ]:
df_fifa.groupBy('Club').sum('value_eur').show(5)

+--------------------+--------------+
|                Club|sum(value_eur)|
+--------------------+--------------+
|       Côte d'Ivoire|             0|
|          Göztepe SK|      43065000|
|CD Everton de Viñ...|      18075000|
|     Shonan Bellmare|       1410000|
|        Salford City|       7905000|
+--------------------+--------------+
only showing top 5 rows



si quisieramos saber cuales son los 5 equipos mas caros hariamos lo siguiente

In [ ]:
df_fifa.groupBy('club') \
  .sum('value_eur') \
  .orderBy('sum(value_eur)', ascending = False) \
  .show(5)

+---------------+--------------+
|           club|sum(value_eur)|
+---------------+--------------+
|    Real Madrid|     897850000|
|   FC Barcelona|     869300000|
|Manchester City|     845745000|
|       Juventus|     735475000|
|      Liverpool|     693265000|
+---------------+--------------+
only showing top 5 rows



Si quisieramos usar más de una función de agregación tendriamos que hacer lo siguiente

In [ ]:
df_fifa.groupBy('club') \
  .agg(sum('value_eur').alias('valor_total'), \
       mean('age').alias('edad_promedio'))\
  .orderBy('valor_total', ascending = False) \
  .show(5)    

+---------------+-----------+------------------+
|           club|valor_total|     edad_promedio|
+---------------+-----------+------------------+
|    Real Madrid|  897850000| 24.90909090909091|
|   FC Barcelona|  869300000|24.060606060606062|
|Manchester City|  845745000|24.333333333333332|
|       Juventus|  735475000|              27.0|
|      Liverpool|  693265000|24.484848484848484|
+---------------+-----------+------------------+
only showing top 5 rows



#### **Crear nuevas columnas**

Podemos crear columnas a partir de los valores de otras, por ejemplo, podríamos querer obtener el valor de los jugadores en pesos.

In [ ]:
df_fifa.withColumn('valor_pesos', df_fifa.value_eur * 25.2).show(5)

+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+-----------+
|       short_name|age|height_cm|weight_kg|nationality|               club|value_eur|preferred_foot|valor_pesos|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+-----------+
|         L. Messi| 32|      170|       72|  Argentina|       FC Barcelona| 95500000|          Left|   2.4066E9|
|Cristiano Ronaldo| 34|      187|       83|   Portugal|           Juventus| 58500000|         Right|   1.4742E9|
|        Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|105500000|         Right|   2.6586E9|
|         J. Oblak| 26|      188|       87|   Slovenia|    Atlético Madrid| 77500000|         Right|    1.953E9|
|        E. Hazard| 28|      175|       74|    Belgium|        Real Madrid| 90000000|         Right|    2.268E9|
+-----------------+---+---------+---------+-----------+-------------------+---------+-----------

#### Muchas funciones a la vez

Si quisieramos saber

## 1.3.3 Consultas SQL

El lector que esté familiarizado con SQL habrá notado que existe cierta similaridad entre las funciones de los DataFrames de PySpark con algunas sentencias de SQL. Este paralelismo llega a tal grado que podemos reemplazar la concatenación de varias funciones con una simple consulta SQL. Por ejemplo, si quisieramos consultar de nuevo cuales son los 5 equipos más caros, solo que esta vez haciendo una consulta SQL, haríamos los siguiente





In [ ]:
# Le dice a spark que guarde este DataFrame como una tabla SQL
df_fifa.registerTempTable('FIFA20')

consulta = '''
  SELECT club, SUM(value_eur) AS valor_total 
  FROM FIFA20
  GROUP BY club
  ORDER BY valor_total DESC
  '''

spark.sql(consulta).show(5)

+---------------+-----------+
|           club|valor_total|
+---------------+-----------+
|    Real Madrid|  897850000|
|   FC Barcelona|  869300000|
|Manchester City|  845745000|
|       Juventus|  735475000|
|      Liverpool|  693265000|
+---------------+-----------+
only showing top 5 rows



Si el lector no está familiarizado con SQL puede referirse a [esta](https://www.w3schools.com/sql/default.asp) y a [esta](https://sqlzoo.net/) fuente. 